In [ ]:
## Import Library to use in this project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## Read data
data = pd.read_excel(r'./restoran.xlsx')

services = pd.DataFrame(data, columns = ['pelayanan'])
services = services.to_numpy().flatten()
foods = pd.DataFrame(data, columns = ['makanan'])
foods = foods.to_numpy().flatten()

In [ ]:
## Rule for Service
 
def bad_service(x):
    if (0 <= x <= 40):
        return 1
    if (40 < x <= 60):
        return -(x-60)/(60-40)
    else:
        return 0

def medium_service(x):
    if (50 < x < 70):
        return (x-50)/(70-50)
    if (70 <= x <= 80):
        return 1
    if (80 < x <= 85):
        return -(x-85)/(85-80)
    else:
        return 0

def good_service(x):
    if (80 < x < 85):
        return (x-80)/(80-85)
    if (85 <= x):
        return 1
    else:
        return 0

In [ ]:
## Rule for Food
    
def bad_food(x):
    if (0 <= x <= 4):
        return 1
    if (4 < x <= 6):
        return -(x-6)/(6-4)
    else:
        return 0
    
def medium_food(x):
    if (4 < x < 5):
        return (x-4)/(5-4)
    if (5 <= x <= 6):
        return 1
    if (6 < x <= 7):
        return -(x-7)/(7-6)
    else:
        return 0
    
def delicious_food(x):
    if (6 < x < 8):
        return (x-6)/(8-6)
    if (8 <= x):
        return 1
    else:
        return 0

In [ ]:
## Fuzzyication

μ_services = []
for service in services:
    μ_service = bad_service(service), medium_service(service), good_service(service)
    μ_services.append(μ_service)
    
μ_foods = []
for food in foods:
    μ_food = bad_food(food), medium_food(food), delicious_food(food)
    μ_foods.append(μ_food)

In [ ]:
## Rule for Qualification Value

def rule(service, food):
    bad_service, medium_service, good_service = service
    bad_food, medium_food, delicious_food = food
    
    low, high = [], []
    if (bad_service and bad_food):
        low.append(min(bad_service, bad_food))
    if (bad_service and medium_food):
        low.append(min(bad_service, medium_food))
    if (bad_service and delicious_food):
        low.append(min(bad_service, delicious_food))

    if (medium_service and bad_food):
        low.append(min(medium_service, bad_food))
    if (medium_service and medium_food):
        high.append(min(medium_service, medium_food))
    if (medium_service and delicious_food):
        high.append(min(medium_service, delicious_food))
    
    if (good_service and bad_food):
        low.append(min(good_service, bad_food))
    if (good_service and medium_food):
        high.append(min(good_service, medium_food))
    if (good_service and delicious_food):
        high.append(min(good_service, delicious_food))
        
        
    if not high:
        high = [0]*len(low)
    if not low:
        low = [0]*len(high)
    
    return max(low), max(high)

In [ ]:
## Inference

qualifications = []
for i in range(len(data)):
    qualifications.append(rule(μ_services[i], μ_foods[i]))

In [ ]:
## ## Rule for Defuzzyication

def low(x):
    if (0 <= x <= 50):
        return 1
    if (50 < x <= 80):
        return -(x-80)/(80-50)
    else:
        return 0
    
def high(x):
    if (70 < x < 85):
        return (x-70)/(85-70)
    if (85 <= x <= 100):
        return 1
    else:
        return 0

In [ ]:
generated_number = [5, 15, 25, 35, 45, 55, 65, 75, 85, 95]
qualification_result = []
for qualification in qualifications:
    total = 0
    total_μ = 0
    for z in generated_number:
        threshold = min(qualification[0], low(z)), min(qualification[1], high(z))
        μ_z = max(threshold[0], threshold[1])
        total = total + (z * μ_z)
        total_μ = total_μ + μ_z
        
    result = total / total_μ    
    qualification_result.append(result)

In [ ]:
df = pd.DataFrame(data)
df['Result'] = qualification_result
df = df.sort_values(by=['Result'], ascending=False).head(20)

df

,id,pelayanan,makanan,Result
69,70,78,8,87.857143
78,79,87,9,87.857143
30,31,74,9,87.857143
59,60,79,6,87.857143
23,24,100,9,87.857143
21,22,79,9,87.857143
68,69,86,10,87.857143
41,42,94,10,87.857143
82,83,69,8,87.761194
53,54,64,10,87.115385


In [ ]:
df = df['id']
df.to_excel("peringkat.xls", index=False)  